**DATA FEATURE ANALYSIS**

In [ ]:

import pandas as pd
import numpy as np

file_path = "/content/drive/MyDrive/Low carbon model/biogas_dataset.csv"
df = pd.read_csv(file_path)
print("Dataset Shape:", df.shape)
print("\nFirst 5 Rows:")
display(df.head())
df.info()
print("\nMissing values per column:")
display(df.isnull().sum())
print("\nNumber of duplicate rows:", df.duplicated().sum())
df = df.drop_duplicates()
print("Shape after removing duplicates:", df.shape)
display(df.describe().T)
display(df.describe(include=['object']))
num_df = df.select_dtypes(include=[np.number])

range_df = pd.DataFrame({
    "Min": num_df.min(),
    "Max": num_df.max(),
    "Range": num_df.max() - num_df.min()
})
display(range_df)
skewness = num_df.skew()
display(skewness)
corr_matrix = num_df.corr()
display(corr_matrix.head())
corr_pairs = corr_matrix.unstack().sort_values(ascending=False)
corr_pairs = corr_pairs[corr_pairs < 1]  # remove self-correlation
display(corr_pairs.head(10))
Q1 = num_df.quantile(0.25)
Q3 = num_df.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = ((num_df < lower_bound) | (num_df > upper_bound)).sum()
display(outliers)
for col in df.select_dtypes(include=['object']).columns:
    print(f"\nColumn: {col}")
    print("Unique count:", df[col].nunique())
    print(df[col].value_counts().head())
summary = df.describe().T
summary_path = "/content/drive/MyDrive/Low carbon model/dataset_summary.csv"
summary.to_csv(summary_path)
print(f"Summary saved to: {summary_path}")



**EXPLORATORY DATA ANALYSIS FOR BIOGAS PRODUCTION DATASET**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

data_path = '/content/drive/MyDrive/Low carbon model/biogas_dataset.csv'
output_path = '/content/drive/MyDrive/Low carbon model/'
df = pd.read_csv(data_path)

print("="*80)
print("BIOGAS PRODUCTION DATASET - EXPLORATORY DATA ANALYSIS")
print("="*80)
print(f"\nDataset shape: {df.shape}")
print(f"Date range: {df['Year'].min()}-{df['Year'].max()}")
print("\n" + "="*80)

print("\n📊 COLUMN INFORMATION:")
print(df.dtypes)

print("\n📊 FIRST FEW ROWS:")
print(df.head())

print("\n📊 BASIC STATISTICS:")
print(df.describe())

# Check for missing values
print("\n📊 MISSING VALUES:")
missing = df.isnull().sum()
print(missing[missing > 0] if any(missing > 0) else "No missing values!")


df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df['Season'] = df['Month'].apply(lambda x:
    'Winter' if x in [12, 1, 2] else
    'Spring' if x in [3, 4, 5] else
    'Summer' if x in [6, 7, 8] else 'Fall')

print("\n📈 Generating distribution plots...")

# Separate variable groups
feedstocks = ['Pig Manure (kg)', 'Kitchen Food Waste (kg)', 'Chicken Litter (kg)',
              'Cassava (kg)', 'Bagasse Feed (kg)', 'Energy Grass (kg)',
              'Banana Shafts (kg)', 'Alcohol Waste (kg)', 'Municipal Residue (kg)',
              'Fish Waste (kg)']

operational = ['Water (L)', 'Diesel (L)', 'Electricity Use (kWh)',
               'C/N Ratio', 'Digester Temp (C)']

climate = ['Temperature (C)', 'Humidity (%)', 'Rainfall (mm)']

target = ['biogas_production']

# Plot distributions - Feedstocks
fig, axes = plt.subplots(5, 2, figsize=(15, 18))
fig.suptitle('Distribution of Feedstock Inputs', fontsize=16, fontweight='bold')
axes = axes.ravel()

for idx, col in enumerate(feedstocks):
    axes[idx].hist(df[col], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    axes[idx].set_title(col, fontweight='bold')
    axes[idx].set_xlabel('Amount (kg)')
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(alpha=0.3)

    # Add statistics
    mean_val = df[col].mean()
    median_val = df[col].median()
    axes[idx].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')
    axes[idx].axvline(median_val, color='green', linestyle='--', linewidth=2, label=f'Median: {median_val:.2f}')
    axes[idx].legend(fontsize=8)

plt.tight_layout()
plt.savefig(output_path + 'EDA_01_feedstock_distributions.png', dpi=300, bbox_inches='tight')
plt.close()

# Plot distributions - Operational & Climate
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
fig.suptitle('Distribution of Operational & Climate Variables', fontsize=16, fontweight='bold')
axes = axes.ravel()

all_vars = operational + climate + target

for idx, col in enumerate(all_vars):
    axes[idx].hist(df[col], bins=50, color='coral', alpha=0.7, edgecolor='black')
    axes[idx].set_title(col, fontweight='bold')
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(alpha=0.3)

    # Add statistics
    mean_val = df[col].mean()
    median_val = df[col].median()
    axes[idx].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')
    axes[idx].axvline(median_val, color='green', linestyle='--', linewidth=2, label=f'Median: {median_val:.2f}')
    axes[idx].legend(fontsize=8)

plt.tight_layout()
plt.savefig(output_path + 'EDA_02_operational_climate_distributions.png', dpi=300, bbox_inches='tight')
plt.close()


print("\n📈 Analyzing biogas production patterns...")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Biogas Production Analysis', fontsize=16, fontweight='bold')

# Distribution
axes[0, 0].hist(df['biogas_production'], bins=60, color='green', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Biogas Production Distribution', fontweight='bold')
axes[0, 0].set_xlabel('Biogas Production (L)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df['biogas_production'].mean(), color='red', linestyle='--',
                   linewidth=2, label=f"Mean: {df['biogas_production'].mean():.2f} L")
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Box plot
axes[0, 1].boxplot(df['biogas_production'], vert=True)
axes[0, 1].set_title('Biogas Production Box Plot', fontweight='bold')
axes[0, 1].set_ylabel('Biogas Production (L)')
axes[0, 1].grid(alpha=0.3)

# Time series
axes[1, 0].plot(df['Date'], df['biogas_production'], alpha=0.6, linewidth=0.5)
axes[1, 0].set_title('Biogas Production Over Time', fontweight='bold')
axes[1, 0].set_xlabel('Date')
axes[1, 0].set_ylabel('Biogas Production (L)')
axes[1, 0].grid(alpha=0.3)

# QQ plot for normality
stats.probplot(df['biogas_production'], dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot (Normality Check)', fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(output_path + 'EDA_03_biogas_production_analysis.png', dpi=300, bbox_inches='tight')
plt.close()

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Temporal Trends in Biogas Production', fontsize=16, fontweight='bold')

# Yearly trend
yearly_avg = df.groupby('Year')['biogas_production'].agg(['mean', 'std', 'count'])
axes[0, 0].plot(yearly_avg.index, yearly_avg['mean'], marker='o', linewidth=2, markersize=8)
axes[0, 0].fill_between(yearly_avg.index,
                        yearly_avg['mean'] - yearly_avg['std'],
                        yearly_avg['mean'] + yearly_avg['std'],
                        alpha=0.3)
axes[0, 0].set_title('Yearly Average Biogas Production', fontweight='bold')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Biogas Production (L)')
axes[0, 0].grid(alpha=0.3)

# Monthly trend
monthly_avg = df.groupby('Month')['biogas_production'].agg(['mean', 'std'])
axes[0, 1].plot(monthly_avg.index, monthly_avg['mean'], marker='o', linewidth=2, markersize=8, color='orange')
axes[0, 1].fill_between(monthly_avg.index,
                        monthly_avg['mean'] - monthly_avg['std'],
                        monthly_avg['mean'] + monthly_avg['std'],
                        alpha=0.3, color='orange')
axes[0, 1].set_title('Monthly Average Biogas Production', fontweight='bold')
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Biogas Production (L)')
axes[0, 1].set_xticks(range(1, 13))
axes[0, 1].grid(alpha=0.3)

# Seasonal comparison
seasonal_data = df.groupby('Season')['biogas_production'].apply(list)
axes[1, 0].boxplot([seasonal_data[s] for s in ['Winter', 'Spring', 'Summer', 'Fall']],
                   labels=['Winter', 'Spring', 'Summer', 'Fall'])
axes[1, 0].set_title('Seasonal Biogas Production', fontweight='bold')
axes[1, 0].set_ylabel('Biogas Production (L)')
axes[1, 0].grid(alpha=0.3)

# Day of month trend
daily_avg = df.groupby('Day')['biogas_production'].mean()
axes[1, 1].plot(daily_avg.index, daily_avg.values, marker='o', linewidth=2, markersize=6, color='green')
axes[1, 1].set_title('Average Biogas Production by Day of Month', fontweight='bold')
axes[1, 1].set_xlabel('Day of Month')
axes[1, 1].set_ylabel('Biogas Production (L)')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(output_path + 'EDA_04_temporal_trends.png', dpi=300, bbox_inches='tight')
plt.close()
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols = [col for col in numerical_cols if col not in ['Year', 'Month', 'Day']]

correlation_matrix = df[numerical_cols].corr()

fig, ax = plt.subplots(figsize=(16, 14))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=0.5, cbar_kws={"shrink": 0.8},
            annot_kws={'size': 8})
plt.title('Correlation Matrix - All Variables', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(output_path + 'EDA_05_correlation_full.png', dpi=300, bbox_inches='tight')
plt.close()

# Correlation with biogas production specifically
biogas_corr = correlation_matrix['biogas_production'].sort_values(ascending=False)
print("\n📊 TOP CORRELATIONS WITH BIOGAS PRODUCTION:")
print(biogas_corr)

# Plot top correlations
fig, ax = plt.subplots(figsize=(10, 12))
biogas_corr_sorted = biogas_corr.drop('biogas_production')  # Remove self-correlation
colors = ['green' if x > 0 else 'red' for x in biogas_corr_sorted.values]
biogas_corr_sorted.plot(kind='barh', color=colors, ax=ax)
ax.set_title('Correlation with Biogas Production', fontsize=14, fontweight='bold')
ax.set_xlabel('Correlation Coefficient')
ax.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(output_path + 'EDA_06_biogas_correlations.png', dpi=300, bbox_inches='tight')
plt.close()
# Calculate total feedstock per observation
df['Total_Feedstock'] = df[feedstocks].sum(axis=1)

# Calculate feedstock proportions
feedstock_props = df[feedstocks].sum() / df['Total_Feedstock'].sum() * 100

# Pie chart
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Feedstock Composition Analysis', fontsize=16, fontweight='bold')

# Pie chart of total usage
colors_pie = plt.cm.Set3(range(len(feedstocks)))
axes[0].pie(feedstock_props.values, labels=feedstock_props.index, autopct='%1.1f%%',
           startangle=90, colors=colors_pie)
axes[0].set_title('Overall Feedstock Composition (%)', fontweight='bold')

# Bar chart of average usage
feedstock_avg = df[feedstocks].mean().sort_values(ascending=False)
axes[1].barh(range(len(feedstock_avg)), feedstock_avg.values, color='steelblue')
axes[1].set_yticks(range(len(feedstock_avg)))
axes[1].set_yticklabels(feedstock_avg.index)
axes[1].set_xlabel('Average Amount (kg)')
axes[1].set_title('Average Feedstock Usage', fontweight='bold')
axes[1].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(output_path + 'EDA_07_feedstock_composition.png', dpi=300, bbox_inches='tight')
plt.close()

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('Climate Variables vs Biogas Production', fontsize=16, fontweight='bold')

# Temperature
axes[0, 0].scatter(df['Temperature (C)'], df['biogas_production'], alpha=0.3, s=10)
axes[0, 0].set_xlabel('Temperature (°C)')
axes[0, 0].set_ylabel('Biogas Production (L)')
axes[0, 0].set_title(f"Correlation: {df['Temperature (C)'].corr(df['biogas_production']):.3f}",
                     fontweight='bold')
axes[0, 0].grid(alpha=0.3)

# Humidity
axes[0, 1].scatter(df['Humidity (%)'], df['biogas_production'], alpha=0.3, s=10, color='green')
axes[0, 1].set_xlabel('Humidity (%)')
axes[0, 1].set_ylabel('Biogas Production (L)')
axes[0, 1].set_title(f"Correlation: {df['Humidity (%)'].corr(df['biogas_production']):.3f}",
                     fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# Rainfall
axes[0, 2].scatter(df['Rainfall (mm)'], df['biogas_production'], alpha=0.3, s=10, color='blue')
axes[0, 2].set_xlabel('Rainfall (mm)')
axes[0, 2].set_ylabel('Biogas Production (L)')
axes[0, 2].set_title(f"Correlation: {df['Rainfall (mm)'].corr(df['biogas_production']):.3f}",
                     fontweight='bold')
axes[0, 2].grid(alpha=0.3)

# Digester Temperature
axes[1, 0].scatter(df['Digester Temp (C)'], df['biogas_production'], alpha=0.3, s=10, color='red')
axes[1, 0].set_xlabel('Digester Temp (°C)')
axes[1, 0].set_ylabel('Biogas Production (L)')
axes[1, 0].set_title(f"Correlation: {df['Digester Temp (C)'].corr(df['biogas_production']):.3f}",
                     fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# C/N Ratio
axes[1, 1].scatter(df['C/N Ratio'], df['biogas_production'], alpha=0.3, s=10, color='purple')
axes[1, 1].set_xlabel('C/N Ratio')
axes[1, 1].set_ylabel('Biogas Production (L)')
axes[1, 1].set_title(f"Correlation: {df['C/N Ratio'].corr(df['biogas_production']):.3f}",
                     fontweight='bold')
axes[1, 1].grid(alpha=0.3)

# Total Feedstock
axes[1, 2].scatter(df['Total_Feedstock'], df['biogas_production'], alpha=0.3, s=10, color='orange')
axes[1, 2].set_xlabel('Total Feedstock (kg)')
axes[1, 2].set_ylabel('Biogas Production (L)')
axes[1, 2].set_title(f"Correlation: {df['Total_Feedstock'].corr(df['biogas_production']):.3f}",
                     fontweight='bold')
axes[1, 2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(output_path + 'EDA_08_climate_biogas_relationships.png', dpi=300, bbox_inches='tight')
plt.close()

# Using IQR method
fig, axes = plt.subplots(3, 4, figsize=(18, 12))
fig.suptitle('Outlier Detection (Box Plots)', fontsize=16, fontweight='bold')
axes = axes.ravel()

selected_vars = feedstocks[:10] + operational[:2]

for idx, col in enumerate(selected_vars):
    if idx < len(axes):
        axes[idx].boxplot(df[col], vert=True)
        axes[idx].set_title(col, fontweight='bold', fontsize=9)
        axes[idx].grid(alpha=0.3)

        # Calculate outliers
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)][col]
        axes[idx].set_ylabel(f'n_outliers: {len(outliers)}', fontsize=8)

plt.tight_layout()
plt.savefig(output_path + 'EDA_09_outlier_detection.png', dpi=300, bbox_inches='tight')
plt.close()

# Select key variables for pairplot
key_vars = ['Pig Manure (kg)', 'Kitchen Food Waste (kg)', 'Digester Temp (C)',
            'Temperature (C)', 'C/N Ratio', 'biogas_production']

# Sample data for faster plotting (optional)
df_sample = df[key_vars].sample(n=min(2000, len(df)), random_state=42)

pairplot = sns.pairplot(df_sample, diag_kind='hist', plot_kws={'alpha': 0.4, 's': 10},
                       diag_kws={'bins': 30, 'edgecolor': 'black'})
pairplot.fig.suptitle('Pairwise Relationships - Key Variables',
                      fontsize=16, fontweight='bold', y=1.01)
plt.savefig(output_path + 'EDA_10_pairwise_relationships.png', dpi=300, bbox_inches='tight')
plt.close()
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Seasonal Effects Analysis', fontsize=16, fontweight='bold')

# Temperature by season
seasonal_temp = [df[df['Season']==s]['Temperature (C)'].values for s in ['Winter', 'Spring', 'Summer', 'Fall']]
axes[0, 0].boxplot(seasonal_temp, labels=['Winter', 'Spring', 'Summer', 'Fall'])
axes[0, 0].set_title('Ambient Temperature by Season', fontweight='bold')
axes[0, 0].set_ylabel('Temperature (°C)')
axes[0, 0].grid(alpha=0.3)

# Humidity by season
seasonal_humidity = [df[df['Season']==s]['Humidity (%)'].values for s in ['Winter', 'Spring', 'Summer', 'Fall']]
axes[0, 1].boxplot(seasonal_humidity, labels=['Winter', 'Spring', 'Summer', 'Fall'])
axes[0, 1].set_title('Humidity by Season', fontweight='bold')
axes[0, 1].set_ylabel('Humidity (%)')
axes[0, 1].grid(alpha=0.3)

# Rainfall by season
seasonal_rainfall = [df[df['Season']==s]['Rainfall (mm)'].values for s in ['Winter', 'Spring', 'Summer', 'Fall']]
axes[1, 0].boxplot(seasonal_rainfall, labels=['Winter', 'Spring', 'Summer', 'Fall'])
axes[1, 0].set_title('Rainfall by Season', fontweight='bold')
axes[1, 0].set_ylabel('Rainfall (mm)')
axes[1, 0].grid(alpha=0.3)

# Biogas by season
seasonal_biogas = [df[df['Season']==s]['biogas_production'].values for s in ['Winter', 'Spring', 'Summer', 'Fall']]
axes[1, 1].boxplot(seasonal_biogas, labels=['Winter', 'Spring', 'Summer', 'Fall'])
axes[1, 1].set_title('Biogas Production by Season', fontweight='bold')
axes[1, 1].set_ylabel('Biogas Production (L)')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(output_path + 'EDA_11_seasonal_effects.png', dpi=300, bbox_inches='tight')
plt.close()
seasonal_summary = df.groupby('Season')[['biogas_production', 'Temperature (C)',
                                         'Humidity (%)', 'Rainfall (mm)']].describe()
print(seasonal_summary)

# Save to CSV
seasonal_summary.to_csv(output_path + 'seasonal_summary_statistics.csv')
summary_report = {
    'Total_Observations': len(df),
    'Date_Range': f"{df['Year'].min()}-{df['Year'].max()}",
    'Biogas_Mean': df['biogas_production'].mean(),
    'Biogas_Std': df['biogas_production'].std(),
    'Biogas_Min': df['biogas_production'].min(),
    'Biogas_Max': df['biogas_production'].max(),
    'Top_Feedstock': df[feedstocks].mean().idxmax(),
    'Top_Feedstock_Avg': df[feedstocks].mean().max(),
    'Strongest_Positive_Correlation': biogas_corr.drop('biogas_production').idxmax(),
    'Strongest_Correlation_Value': biogas_corr.drop('biogas_production').max(),
    'Average_Temperature': df['Temperature (C)'].mean(),
    'Average_Humidity': df['Humidity (%)'].mean(),
    'Average_CN_Ratio': df['C/N Ratio'].mean(),
}

summary_df = pd.DataFrame([summary_report])
summary_df.to_csv(output_path + 'EDA_summary_report.csv', index=False)

print("\n" + "="*80)
print("✅ EXPLORATORY DATA ANALYSIS COMPLETE!")
print("="*80)
print(f"\n📁 All outputs saved to: {output_path}")
print("\nGenerated files:")
print("  - EDA_01_feedstock_distributions.png")
print("  - EDA_02_operational_climate_distributions.png")
print("  - EDA_03_biogas_production_analysis.png")
print("  - EDA_04_temporal_trends.png")
print("  - EDA_05_correlation_full.png")
print("  - EDA_06_biogas_correlations.png")
print("  - EDA_07_feedstock_composition.png")
print("  - EDA_08_climate_biogas_relationships.png")
print("  - EDA_09_outlier_detection.png")
print("  - EDA_10_pairwise_relationships.png")
print("  - EDA_11_seasonal_effects.png")
print("  - seasonal_summary_statistics.csv")
print("  - EDA_summary_report.csv")

print("\n" + "="*80)
print("📊 KEY FINDINGS:")
print("="*80)
for key, value in summary_report.items():
    print(f"{key}: {value}")

**BIOGAS ML PROOF-OF-CONCEPT TEST**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("🧪 BIOGAS ML PROOF-OF-CONCEPT TEST")
print("="*80)


data_path = '/content/drive/MyDrive/Low carbon model/biogas_dataset.csv'
output_path = '/content/drive/MyDrive/Low carbon model/'

df = pd.read_csv(data_path)
print(f"\n✅ Data loaded: {df.shape[0]} observations, {df.shape[1]} features")

# Define feature groups
feedstocks = ['Pig Manure (kg)', 'Kitchen Food Waste (kg)', 'Chicken Litter (kg)',
              'Cassava (kg)', 'Bagasse Feed (kg)', 'Energy Grass (kg)',
              'Banana Shafts (kg)', 'Alcohol Waste (kg)', 'Municipal Residue (kg)',
              'Fish Waste (kg)']

operational = ['Water (L)', 'Diesel (L)', 'Electricity Use (kWh)',
               'C/N Ratio', 'Digester Temp (C)']

climate = ['Temperature (C)', 'Humidity (%)', 'Rainfall (mm)']

# Combine all features
all_features = feedstocks + operational + climate
target = 'biogas_production'

# Prepare X and y
X = df[all_features].copy()
y = df[target].copy()

print(f"\n✅ Features prepared: {X.shape[1]} input features")
print(f"   - {len(feedstocks)} feedstocks")
print(f"   - {len(operational)} operational variables")
print(f"   - {len(climate)} climate variables")

# ============================================================================
# 3. TRAIN/TEST SPLIT (Temporal)
# ============================================================================

# Use 80% for training, 20% for testing (temporal split)
split_idx = int(0.8 * len(df))

X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f"\n✅ Data split:")
print(f"   - Training: {len(X_train)} samples ({100*len(X_train)/len(X):.1f}%)")
print(f"   - Testing: {len(X_test)} samples ({100*len(X_test)/len(X):.1f}%)")

print("\n" + "="*80)
print("🤖 TESTING ML MODELS...")
print("="*80)

models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=15,
                                          random_state=42, n_jobs=-1),
}

# Try to import advanced models (skip if not installed)
try:
    from xgboost import XGBRegressor
    models['XGBoost'] = XGBRegressor(n_estimators=100, max_depth=8,
                                    learning_rate=0.1, random_state=42, n_jobs=-1)
    print("✅ XGBoost available")
except ImportError:
    print("⚠️  XGBoost not installed (will use RF only)")

try:
    from lightgbm import LGBMRegressor
    models['LightGBM'] = LGBMRegressor(n_estimators=100, max_depth=8,
                                      learning_rate=0.1, random_state=42, n_jobs=-1,
                                      verbose=-1)
    print("✅ LightGBM available")
except ImportError:
    print("⚠️  LightGBM not installed (will use RF only)")

# Train and evaluate each model
results = {}

for model_name, model in models.items():
    print(f"\n{'─'*60}")
    print(f"Training {model_name}...")

    # Train
    model.fit(X_train, y_train)

    # Predict
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)

    # Calculate MAPE
    mape = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100

    results[model_name] = {
        'model': model,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'test_rmse': test_rmse,
        'test_mae': test_mae,
        'mape': mape,
        'predictions': y_test_pred
    }

    print(f"✅ {model_name} Results:")
    print(f"   Training R²:  {train_r2:.4f}")
    print(f"   Testing R²:   {test_r2:.4f}")
    print(f"   RMSE:         {test_rmse:.4f} L")
    print(f"   MAE:          {test_mae:.4f} L")
    print(f"   MAPE:         {mape:.2f}%")

    # Check for overfitting
    overfit_gap = train_r2 - test_r2
    if overfit_gap > 0.1:
        print(f"   ⚠️  Overfitting detected (gap: {overfit_gap:.4f})")
    else:
        print(f"   ✅ No significant overfitting (gap: {overfit_gap:.4f})")

best_model_name = max(results, key=lambda k: results[k]['test_r2'])
best_model = results[best_model_name]['model']
best_r2 = results[best_model_name]['test_r2']

print("\n" + "="*80)
print(f"🏆 BEST MODEL: {best_model_name} (R² = {best_r2:.4f})")
print("="*80)

print("\n📊 Generating prediction visualizations...")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle(f'Model Performance: {best_model_name}', fontsize=16, fontweight='bold')

# Plot 1: Observed vs Predicted
y_test_pred = results[best_model_name]['predictions']
axes[0, 0].scatter(y_test, y_test_pred, alpha=0.5, s=20)
axes[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
                'r--', linewidth=2, label='Perfect Prediction')
axes[0, 0].set_xlabel('Observed Biogas Production (L)', fontweight='bold')
axes[0, 0].set_ylabel('Predicted Biogas Production (L)', fontweight='bold')
axes[0, 0].set_title(f'Observed vs Predicted (R² = {best_r2:.4f})', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Plot 2: Residuals
residuals = y_test - y_test_pred
axes[0, 1].scatter(y_test_pred, residuals, alpha=0.5, s=20)
axes[0, 1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Predicted Biogas Production (L)', fontweight='bold')
axes[0, 1].set_ylabel('Residuals (L)', fontweight='bold')
axes[0, 1].set_title('Residual Plot', fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# Plot 3: Residual distribution
axes[1, 0].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1, 0].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Residuals (L)', fontweight='bold')
axes[1, 0].set_ylabel('Frequency', fontweight='bold')
axes[1, 0].set_title(f'Residual Distribution (Mean: {residuals.mean():.2f} L)',
                     fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# Plot 4: Time series of predictions
test_indices = np.arange(len(y_test))
axes[1, 1].plot(test_indices, y_test.values, label='Observed', alpha=0.7, linewidth=1)
axes[1, 1].plot(test_indices, y_test_pred, label='Predicted', alpha=0.7, linewidth=1)
axes[1, 1].set_xlabel('Test Sample Index', fontweight='bold')
axes[1, 1].set_ylabel('Biogas Production (L)', fontweight='bold')
axes[1, 1].set_title('Time Series: Observed vs Predicted', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(output_path + 'POC_model_performance.png', dpi=300, bbox_inches='tight')
plt.close()

print("✅ Saved: POC_model_performance.png")
print("\n📊 Analyzing feature importance...")

# Get feature importance
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    feature_importance_df = pd.DataFrame({
        'Feature': all_features,
        'Importance': importances
    }).sort_values('Importance', ascending=False)

    print("\n🔝 TOP 10 MOST IMPORTANT FEATURES:")
    print(feature_importance_df.head(10).to_string(index=False))

    # Plot feature importance
    fig, ax = plt.subplots(figsize=(10, 8))
    top_features = feature_importance_df.head(15)
    ax.barh(range(len(top_features)), top_features['Importance'].values)
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['Feature'].values)
    ax.set_xlabel('Importance', fontweight='bold')
    ax.set_title(f'Top 15 Feature Importances ({best_model_name})',
                 fontsize=14, fontweight='bold')
    ax.grid(alpha=0.3, axis='x')
    plt.tight_layout()
    plt.savefig(output_path + 'POC_feature_importance.png', dpi=300, bbox_inches='tight')
    plt.close()

    print("✅ Saved: POC_feature_importance.png")

try:
    import shap
    print("\n📊 Testing SHAP explainability...")

    # Use a small sample for quick test (SHAP can be slow)
    X_sample = X_test.sample(min(500, len(X_test)), random_state=42)

    # Create explainer
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer.shap_values(X_sample)

    # Summary plot
    fig, ax = plt.subplots(figsize=(10, 8))
    shap.summary_plot(shap_values, X_sample, plot_type="bar", show=False, max_display=15)
    plt.title('SHAP Feature Importance', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(output_path + 'POC_shap_importance.png', dpi=300, bbox_inches='tight')
    plt.close()

    print("✅ SHAP analysis works! Saved: POC_shap_importance.png")

except ImportError:
    print("⚠️  SHAP not installed - install with: pip install shap")
except Exception as e:
    print(f"⚠️  SHAP test failed: {e}")

print("\n📊 Testing impact of different feature groups...")

feature_groups = {
    'Feedstocks Only': feedstocks,
    'Feedstocks + Operational': feedstocks + operational,
    'Feedstocks + Climate': feedstocks + climate,
    'All Features': all_features
}

group_results = {}

for group_name, features in feature_groups.items():
    X_group_train = X_train[features]
    X_group_test = X_test[features]

    # Train simple RF model
    model_group = RandomForestRegressor(n_estimators=50, max_depth=15,
                                       random_state=42, n_jobs=-1)
    model_group.fit(X_group_train, y_train)

    # Evaluate
    y_pred_group = model_group.predict(X_group_test)
    r2_group = r2_score(y_test, y_pred_group)

    group_results[group_name] = r2_group
    print(f"  {group_name:30s} R² = {r2_group:.4f}")

print("\n" + "="*80)
print("📋 PROOF-OF-CONCEPT SUMMARY")
print("="*80)

print(f"\n✅ DATA VALIDATION:")
print(f"   - Dataset size: {len(df):,} observations")
print(f"   - Features: {len(all_features)}")
print(f"   - Target range: {y.min():.2f} - {y.max():.2f} L")
print(f"   - No missing values: {X.isnull().sum().sum() == 0}")

print(f"\n✅ MODEL PERFORMANCE:")
print(f"   - Best model: {best_model_name}")
print(f"   - Test R²: {best_r2:.4f}")
print(f"   - RMSE: {results[best_model_name]['test_rmse']:.4f} L")
print(f"   - MAPE: {results[best_model_name]['mape']:.2f}%")

if 'feature_importance_df' in locals():
    top3 = feature_importance_df.head(3)['Feature'].tolist()
    print(f"\n✅ TOP 3 FEATURES:")
    for i, feat in enumerate(top3, 1):
        imp = feature_importance_df[feature_importance_df['Feature']==feat]['Importance'].values[0]
        print(f"   {i}. {feat} ({imp:.4f})")

print(f"\n✅ FEATURE GROUP COMPARISON:")
for group_name, r2 in group_results.items():
    print(f"   {group_name:30s} R² = {r2:.4f}")

# Calculate climate contribution
climate_contribution = group_results['All Features'] - group_results['Feedstocks + Operational']
print(f"\n📊 CLIMATE CONTRIBUTION: ΔR² = {climate_contribution:.4f}")
if abs(climate_contribution) < 0.02:
    print("   ✅ Climate has minimal impact (supports your hypothesis!)")
else:
    print("   ⚠️  Climate has noticeable impact")

print("\n" + "="*80)
print("🎯 DECISION: Should you proceed with full framework?")
print("="*80)

if best_r2 > 0.85:
    print("✅ YES! Strong performance (R² > 0.85)")
    print("   → Proceed with hierarchical framework")
    print("   → SHAP analysis will provide valuable insights")
    print("   → Optimization will be effective")
elif best_r2 > 0.75:
    print("✅ PROBABLY YES. Good performance (R² > 0.75)")
    print("   → Consider feature engineering to boost performance")
    print("   → Test more advanced models (CatBoost, ensemble)")
elif best_r2 > 0.65:
    print("⚠️  MAYBE. Moderate performance (R² > 0.65)")
    print("   → Investigate data quality issues")
    print("   → Try non-linear feature transformations")
    print("   → Consider domain knowledge for feature engineering")
else:
    print("❌ NOT YET. Low performance (R² < 0.65)")
    print("   → Review data quality and preprocessing")
    print("   → Check for missing important features")
    print("   → Consider mechanistic features")

print("\n" + "="*80)
print("📁 OUTPUTS SAVED:")
print("="*80)
print("  1. POC_model_performance.png")
print("  2. POC_feature_importance.png")
if 'shap' in dir():
    print("  3. POC_shap_importance.png")
print("\n✅ PROOF-OF-CONCEPT COMPLETE!")
print("="*80)

**FULL MODEL BIOGAS ML **

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import TimeSeriesSplit
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import itertools
import networkx as nx

data_path = '/content/drive/MyDrive/Low carbon model/biogas_dataset.csv'
output_path = '/content/drive/MyDrive/Low carbon model/'

df = pd.read_csv(data_path)

# Define features
feedstocks = ['Pig Manure (kg)', 'Kitchen Food Waste (kg)', 'Chicken Litter (kg)',
              'Cassava (kg)', 'Bagasse Feed (kg)', 'Energy Grass (kg)',
              'Banana Shafts (kg)', 'Alcohol Waste (kg)', 'Municipal Residue (kg)',
              'Fish Waste (kg)']

operational = ['Water (L)', 'Diesel (L)', 'Electricity Use (kWh)',
               'C/N Ratio', 'Digester Temp (C)']

climate = ['Temperature (C)', 'Humidity (%)', 'Rainfall (mm)']

all_features = feedstocks + operational + climate
target = 'biogas_production'

X = df[all_features]
y = df[target]

# Temporal split
split_idx = int(0.8 * len(df))
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# Optimized hyperparameters (can tune further)
model = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=10,
    num_leaves=31,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

model.fit(X_train, y_train)

# Evaluate
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_mae = mean_absolute_error(y_test, y_pred_test)
test_mape = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100

print(f"\n✅ MODEL PERFORMANCE:")
print(f"   Training R²:  {train_r2:.4f}")
print(f"   Testing R²:   {test_r2:.4f}")
print(f"   RMSE:         {test_rmse:.4f} L")
print(f"   MAE:          {test_mae:.4f} L")
print(f"   MAPE:         {test_mape:.2f}%")


# Create SHAP explainer
explainer = shap.TreeExplainer(model)

# Calculate SHAP values for test set (use sample if too large)
sample_size = min(2000, len(X_test))
X_shap = X_test.sample(sample_size, random_state=42)
shap_values = explainer.shap_values(X_shap)
shap_interaction_values = explainer.shap_interaction_values(X_shap)

print(f"✅ SHAP values calculated for {sample_size} samples")

# Calculate mean absolute SHAP values
mean_abs_shap = np.abs(shap_values).mean(axis=0)
feature_importance_df = pd.DataFrame({
    'Feature': all_features,
    'SHAP_Importance': mean_abs_shap
}).sort_values('SHAP_Importance', ascending=False)

print("\n🔝 TOP 10 FEATURES (by SHAP importance):")
print(feature_importance_df.head(10).to_string(index=False))

# FIGURE 1: SHAP Summary Plot (Beeswarm)
fig, ax = plt.subplots(figsize=(10, 10))
shap.summary_plot(shap_values, X_shap, plot_type="dot", show=False, max_display=18)
plt.title('SHAP Feature Importance Summary', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(output_path + 'SHAP_01_summary_beeswarm.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Saved: SHAP_01_summary_beeswarm.png")

# FIGURE 2: SHAP Bar Plot
fig, ax = plt.subplots(figsize=(10, 10))
shap.summary_plot(shap_values, X_shap, plot_type="bar", show=False, max_display=18)
plt.title('Mean Absolute SHAP Values', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(output_path + 'SHAP_02_bar_importance.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Saved: SHAP_02_bar_importance.png")

print("\n📊 Generating partial dependence plots...")

top_6_features = feature_importance_df.head(6)['Feature'].tolist()

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('SHAP Dependence Plots - Top 6 Features', fontsize=16, fontweight='bold')
axes = axes.ravel()

for idx, feature in enumerate(top_6_features):
    feature_idx = all_features.index(feature)
    shap.dependence_plot(
        feature_idx,
        shap_values,
        X_shap,
        ax=axes[idx],
        show=False
    )
    axes[idx].set_title(feature, fontweight='bold')

plt.tight_layout()
plt.savefig(output_path + 'SHAP_03_dependence_plots.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Saved: SHAP_03_dependence_plots.png")

print("\n" + "="*80)
print("🔬 FEEDSTOCK SYNERGY NETWORK ANALYSIS (NOVEL)")
print("="*80)

# Calculate mean absolute interaction values for all feedstock pairs
feedstock_indices = [all_features.index(f) for f in feedstocks]
n_feedstocks = len(feedstocks)

# Initialize interaction matrix
interaction_matrix = np.zeros((n_feedstocks, n_feedstocks))

for i in range(n_feedstocks):
    for j in range(n_feedstocks):
        if i != j:
            # Mean absolute interaction across all samples
            interaction_matrix[i, j] = np.abs(
                shap_interaction_values[:, feedstock_indices[i], feedstock_indices[j]]
            ).mean()

# Create DataFrame
interaction_df = pd.DataFrame(
    interaction_matrix,
    index=feedstocks,
    columns=feedstocks
)

print("\n📊 FEEDSTOCK INTERACTION MATRIX:")
print(interaction_df.round(4))

# Save to CSV
interaction_df.to_csv(output_path + 'feedstock_interaction_matrix.csv')
print("✅ Saved: feedstock_interaction_matrix.csv")

# FIGURE 3: Interaction Heatmap
fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(interaction_df, annot=True, fmt='.3f', cmap='YlOrRd',
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Feedstock Synergy Interaction Matrix (SHAP)',
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(output_path + 'SHAP_04_interaction_heatmap.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Saved: SHAP_04_interaction_heatmap.png")
# Create network
G = nx.Graph()

# Add nodes
for feedstock in feedstocks:
    G.add_node(feedstock)

# Calculate threshold for significant interactions (top 30%)
all_interactions = interaction_matrix[np.triu_indices_from(interaction_matrix, k=1)]
threshold = np.percentile(all_interactions, 70)

print(f"✅ Interaction threshold: {threshold:.4f}")

# Add edges for significant interactions
significant_pairs = []
for i in range(n_feedstocks):
    for j in range(i+1, n_feedstocks):
        interaction_strength = interaction_matrix[i, j]
        if interaction_strength > threshold:
            G.add_edge(
                feedstocks[i],
                feedstocks[j],
                weight=interaction_strength
            )
            significant_pairs.append((feedstocks[i], feedstocks[j], interaction_strength))

print(f"✅ Identified {len(significant_pairs)} significant synergies")

# Calculate network metrics
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)

# Identify keystone feedstocks
keystone_feedstocks = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)

print("\n🔑 KEYSTONE FEEDSTOCKS (by network centrality):")
for i, (feedstock, centrality) in enumerate(keystone_feedstocks[:5], 1):
    print(f"   {i}. {feedstock}: {centrality:.3f}")

fig, ax = plt.subplots(figsize=(14, 12))

# Layout
pos = nx.spring_layout(G, k=2, iterations=50, seed=42)

# Node sizes based on main effect (mean absolute SHAP)
feedstock_shap_importance = {
    f: feature_importance_df[feature_importance_df['Feature']==f]['SHAP_Importance'].values[0]
    for f in feedstocks if f in feature_importance_df['Feature'].values
}
node_sizes = [feedstock_shap_importance.get(node, 0) * 100 for node in G.nodes()]

# Node colors based on centrality
node_colors = [degree_centrality[node] for node in G.nodes()]

# Edge widths based on interaction strength
edge_widths = [G[u][v]['weight'] * 20 for u, v in G.edges()]

# Draw network
nodes = nx.draw_networkx_nodes(
    G, pos,
    node_size=node_sizes,
    node_color=node_colors,
    cmap=plt.cm.viridis,
    alpha=0.9,
    ax=ax
)

nx.draw_networkx_edges(
    G, pos,
    width=edge_widths,
    alpha=0.6,
    edge_color='gray',
    ax=ax
)

nx.draw_networkx_labels(
    G, pos,
    font_size=9,
    font_weight='bold',
    ax=ax
)

# Add colorbar
plt.colorbar(nodes, label='Network Centrality', ax=ax, shrink=0.8)

ax.set_title(
    'Feedstock Synergy Network\n(Node size = main effect | Edge width = interaction strength | Color = centrality)',
    fontsize=14, fontweight='bold', pad=20
)
ax.axis('off')
plt.tight_layout()
plt.savefig(output_path + 'SHAP_05_synergy_network.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Saved: SHAP_05_synergy_network.png")
print("\n🔝 TOP 10 SYNERGISTIC FEEDSTOCK PAIRS:")
significant_pairs_sorted = sorted(significant_pairs, key=lambda x: x[2], reverse=True)
for i, (f1, f2, strength) in enumerate(significant_pairs_sorted[:10], 1):
    print(f"   {i}. {f1} × {f2}: {strength:.4f}")

# Save to CSV
synergy_pairs_df = pd.DataFrame(
    significant_pairs_sorted,
    columns=['Feedstock_1', 'Feedstock_2', 'Interaction_Strength']
)
synergy_pairs_df.to_csv(output_path + 'feedstock_synergy_pairs.csv', index=False)
print("✅ Saved: feedstock_synergy_pairs.csv")
print("\n" + "="*80)
print("🌡️ CLIMATE RESILIENCE ANALYSIS")
print("="*80)

# Extract SHAP values for climate variables
climate_indices = [all_features.index(c) for c in climate]
climate_shap = shap_values[:, climate_indices]

# Calculate contribution
climate_shap_importance = np.abs(climate_shap).mean(axis=0)
total_shap_importance = np.abs(shap_values).mean(axis=0).sum()
climate_contribution_pct = (climate_shap_importance.sum() / total_shap_importance) * 100

print(f"\n📊 CLIMATE VARIABLE CONTRIBUTIONS:")
for i, climate_var in enumerate(climate):
    pct = (climate_shap_importance[i] / total_shap_importance) * 100
    print(f"   {climate_var:20s}: {pct:.2f}%")

print(f"\n✅ Total climate contribution: {climate_contribution_pct:.2f}%")

# FIGURE 5: Climate vs Feedstock Contributions
fig, ax = plt.subplots(figsize=(10, 6))

categories = ['Feedstocks', 'Operational', 'Climate']
feedstock_shap = np.abs(shap_values[:, [all_features.index(f) for f in feedstocks]]).mean()
operational_shap = np.abs(shap_values[:, [all_features.index(o) for o in operational]]).mean()
climate_shap_total = np.abs(shap_values[:, climate_indices]).mean()

contributions = [feedstock_shap, operational_shap, climate_shap_total]
colors = ['#2ecc71', '#3498db', '#e74c3c']

bars = ax.bar(categories, contributions, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

ax.set_ylabel('Mean |SHAP Value|', fontweight='bold', fontsize=12)
ax.set_title('Feature Group Contributions to Biogas Production',
             fontsize=14, fontweight='bold')
ax.grid(alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.3f}',
            ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.savefig(output_path + 'SHAP_06_feature_group_contributions.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Saved: SHAP_06_feature_group_contributions.png")
print("\n" + "="*80)
print("🔍 LOCAL EXPLANATION EXAMPLES")
print("="*80)

# Find interesting cases
residuals = y_test.values - y_pred_test
residuals_indexed = pd.Series(residuals, index=y_test.index)

# Case 1: Best prediction
best_idx = residuals_indexed.abs().idxmin()
best_idx_in_shap = X_shap.index.get_loc(best_idx) if best_idx in X_shap.index else 0

# Case 2: Worst prediction
worst_idx = residuals_indexed.abs().idxmax()
worst_idx_in_shap = X_shap.index.get_loc(worst_idx) if worst_idx in X_shap.index else 1

# Case 3: High yield
high_yield_idx = y_test.idxmax()
high_idx_in_shap = X_shap.index.get_loc(high_yield_idx) if high_yield_idx in X_shap.index else 2

cases = [
    ('Best Prediction', best_idx_in_shap),
    ('Worst Prediction', worst_idx_in_shap),
    ('Highest Yield', high_idx_in_shap)
]

fig, axes = plt.subplots(3, 1, figsize=(12, 15))

for idx, (case_name, case_idx) in enumerate(cases):
    shap.waterfall_plot(
        shap.Explanation(
            values=shap_values[case_idx],
            base_values=explainer.expected_value,
            data=X_shap.iloc[case_idx],
            feature_names=all_features
        ),
        max_display=15,
        show=False
    )
    axes[idx].set_title(f'{case_name}', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.savefig(output_path + 'SHAP_07_local_explanations.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Saved: SHAP_07_local_explanations.png")

print("\n" + "="*80)
print("📋 COMPREHENSIVE ANALYSIS SUMMARY")
print("="*80)

summary_report = {
    'Model': 'LightGBM',
    'Test_R2': test_r2,
    'Test_RMSE_L': test_rmse,
    'Test_MAE_L': test_mae,
    'Test_MAPE_pct': test_mape,
    'Top_Feature': feature_importance_df.iloc[0]['Feature'],
    'Top_Feature_Importance': feature_importance_df.iloc[0]['SHAP_Importance'],
    'Climate_Contribution_pct': climate_contribution_pct,
    'Keystone_Feedstock': keystone_feedstocks[0][0],
    'Keystone_Centrality': keystone_feedstocks[0][1],
    'N_Significant_Synergies': len(significant_pairs),
}

print("\n✅ KEY METRICS:")
for key, value in summary_report.items():
    print(f"   {key}: {value}")

# Save summary
summary_df = pd.DataFrame([summary_report])
summary_df.to_csv(output_path + 'comprehensive_analysis_summary.csv', index=False)
print("\n✅ Saved: comprehensive_analysis_summary.csv")

print("\n" + "="*80)
print("✅ COMPREHENSIVE ANALYSIS COMPLETE!")
print("="*80)

print("\n📁 GENERATED FILES:")
files = [
    'SHAP_01_summary_beeswarm.png',
    'SHAP_02_bar_importance.png',
    'SHAP_03_dependence_plots.png',
    'SHAP_04_interaction_heatmap.png',
    'SHAP_05_synergy_network.png ⭐ KEY FIGURE',
    'SHAP_06_feature_group_contributions.png',
    'SHAP_07_local_explanations.png',
    'feedstock_interaction_matrix.csv',
    'feedstock_synergy_pairs.csv',
    'comprehensive_analysis_summary.csv'
]

for i, file in enumerate(files, 1):
    print(f"  {i}. {file}")

print("\n" + "="*80)
